In [4]:
import time
import datetime as dt
import numpy as np
import pyvisa

In [5]:
# rm = visa.ResourceManager('C:\\Windows\\System32\\visa32.dll')

#save_path = r"C:/Users/IPMU\Documents/Python Scripts/devinit/"
save_path = r"C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/"

# look up instruments
rm = pyvisa.ResourceManager()
print("resource manager : ", rm)
rl = rm.list_resources()
print("List of resources: \n", rl)

resource manager :  Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
List of resources: 
 ('ASRL1::INSTR', 'ASRL3::INSTR', 'GPIB1::12::INSTR', 'GPIB2::16::INSTR', 'GPIB3::2::INSTR')


In [6]:
for i,name in enumerate(rl):
    if "GPIB" in name:
        my_instrument = rm.open_resource(name)
        print("Resource[",i,"]:", name, "\n", my_instrument.query("*IDN?"))

Resource[ 2 ]: GPIB1::12::INSTR 
 LSCI,MODEL218S,LSB2EH,041813

Resource[ 3 ]: GPIB2::16::INSTR 
 KEITHLEY INSTRUMENTS INC.,MODEL 2700,4096641,B10  /A02  

Resource[ 4 ]: GPIB3::2::INSTR 
 NF Corporation,WF1948,9182457,Ver1.40



In [7]:
KT2700 = [rm.open_resource(rl[3])]
filename = save_path + dt.datetime.now().strftime("%Y%m%d%H%M%S") + "_MULTIMETER_KT2700.csv"
print(filename)

columnname = ['abstime','reltime','Voltage[V]']
columnname = ['reltime','Voltage[V]']

try:
    with open(filename, mode="a") as f:
        print(*columnname, sep=", ", file=f)
except:
    pass

REFRESHRATE = 0.1  # [sec] for frequency of daq

C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/20250221110613_MULTIMETER_KT2700.csv


In [8]:
KT2700[0].write('*RST')
KT2700[0].write(':INITiate:CONTinuous OFF;:ABORt')
KT2700[0].write(':SENSe:VOLTage:DC:RANGe:AUTO ON')
KT2700[0].write(':SENSe:FUNCtion "VOLTage:DC"')
KT2700[0].write(':SENSe:VOLTage:DC:NPLC 0.01') # Use lowest NPLC setting for fastest readings.
KT2700[0].write(':SYSTem:AZERo:STATe OFF') #  Disable autozero to increase speed, but may cause drift over time.
KT2700[0].write(':SENSe:VOLTage:DC:AVERage:STATe OFF') # Turn off filter for speed.
KT2700[0].write(':TRIGger:COUNt 1')
KT2700[0].write('READ?')
buf_str = KT2700[0].read()


In [5]:
def getVotage(self):
    self.write('READ?')
    buf_str = self.read()
    buf_str = buf_str.strip()  # 改行文字を削除
    buf_str_list = buf_str.split(',')  # カンマで分割
    buf_float = float(buf_str_list[0][:-3]) # [:-3] is coresponding to "VDC"
    return buf_float

In [10]:
StarTime = time.perf_counter()

while True:

    #CurrentAbsTime = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    CurrentRerTime = time.perf_counter() - StarTime
    #print(time.perf_counter())
    #arr = [CurrentAbsTime, CurrentRerTime]

    arr = [CurrentRerTime]
    
    buf = getVotage(KT2700[0])
    arr.append(buf)

    print(arr)
    time.sleep(REFRESHRATE)

    try:
        with open(filename, mode="a") as f:
            print(*arr, sep=", ", file=f)

    except FileNotFoundError:

        print("FileNotFoundError... :'-(")
        filename = save_path + dt.datetime.now().strftime("%Y%m%d%H%M%S") + ".csv"
        with open(filename, mode="a") as f:
            print(*columnname, sep=", ", file=f)
            print(*arr, sep=", ", file=f)

[8.629999999953952e-05, 0.0190714579]
[0.1507628999999966, -0.0139528811]
[0.28528549999999697, 0.0138909761]
[0.427593599999998, 0.00435504876]
[0.5622969999999974, 0.00687035918]
[0.7063502999999969, 0.002609357]
[0.8379707999999972, -0.0121051138]
[0.9746555999999984, -0.0603007451]
[1.1052378999999988, -0.0132119386]
[1.2488310999999968, -0.0305196978]
[1.3839687999999981, -0.0247137416]
[1.5257359999999984, -0.0297776088]
[1.668893899999997, -0.0620865747]
[1.8069390999999975, -0.0404432714]
[1.9412952999999966, -0.0476714708]
[2.090041499999998, -0.0516766869]
[2.2371269, -0.0531723313]
[2.385258499999999, -0.0440642536]
[2.5236326999999967, -0.0508462787]
[2.6602917999999995, -0.0587638058]
[2.7937908999999976, -0.0442596897]
[2.9454284999999985, -0.0240829103]
[3.087308099999998, -0.0168455355]
[3.2207674999999973, -0.0251289457]
[3.3685674999999975, -0.0364690274]
[3.5108578999999978, -0.0704525635]
[3.644529200000001, -0.0406130254]
[3.7818427999999997, -0.0476313271]
[3.9177

KeyboardInterrupt: 

In [1]:
import lib_KT2700
rm, rl = lib_KT2700.query()
KT2700 = dict()
KT2700["save_path"] = r"C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/"
KT2700["columnnames"] = ["reltime", "Voltage"]
KT2700["filename"] = lib_KT2700.create_csv(
    KT2700["save_path"], KT2700["columnnames"]
)
KT2700["instrument"] = rm.open_resource(rl[3])
lib_KT2700.initialize(instrument=KT2700["instrument"])

resource manager :  Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
List of resources: 
 ('ASRL1::INSTR', 'ASRL3::INSTR', 'GPIB1::12::INSTR', 'GPIB2::16::INSTR', 'GPIB3::2::INSTR')
Resource[ 2 ]: GPIB1::12::INSTR 
 LSCI,MODEL218S,LSB2EH,041813

Resource[ 3 ]: GPIB2::16::INSTR 
 KEITHLEY INSTRUMENTS INC.,MODEL 2700,4096641,B10  /A02  

Resource[ 4 ]: GPIB3::2::INSTR 
 NF Corporation,WF1948,9182457,Ver1.40

Generated filename: C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/20250221195801_MULTIMETER_KT2700.csv
+9.08757187E-03VDC,+287.213SECS,+01024RDNG#

KT2700 initialization complete.


In [ ]:
# Example usage:
import lib_time

if __name__ == '__main__':
    # Set the sampling rate in samples per second (inverse of the refresh rate)
    sampling_rate = 100  # For example, 1 sample per second (adjust as needed)
    # Create a time provider generator with the specified sampling rate
    tg = lib_time.time_generator(sampling_rate)
    
    # Start continuous data acquisition, passing in the time provider generator
    lib_KT2700.DataAcquisition(tg, KT2700["filename"], KT2700["save_path"], KT2700["save_path"],KT2700["instrument"])

KeyboardInterrupt: 

In [6]:
import lib_KT2700
import lib_time

rm, rl = lib_KT2700.query()
KT2700 = dict()
KT2700["save_path"] = r"C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/"
KT2700["columnnames"] = ["reltime", "Voltage"]
KT2700["filename"] = lib_KT2700.create_csv(
    KT2700["save_path"], KT2700["columnnames"]
)
KT2700["instrument"] = rm.open_resource(rl[3])
lib_KT2700.initialize(instrument=KT2700["instrument"])

resource manager :  Resource Manager of Visa Library at C:\WINDOWS\system32\visa32.dll
List of resources: 
 ('ASRL1::INSTR', 'ASRL3::INSTR', 'GPIB1::12::INSTR', 'GPIB2::16::INSTR', 'GPIB3::2::INSTR')
Resource[ 2 ]: GPIB1::12::INSTR 
 LSCI,MODEL218S,LSB2EH,041813

Resource[ 3 ]: GPIB2::16::INSTR 
 KEITHLEY INSTRUMENTS INC.,MODEL 2700,4096641,B10  /A02  

Resource[ 4 ]: GPIB3::2::INSTR 
 NF Corporation,WF1948,9182457,Ver1.40

Generated filename: C:/Users/IPMU/Desktop/2025-gripper-run3/Keithley2700/20250221211015_MULTIMETER_KT2700.csv
-2.65377127E-02VDC,+4621.939SECS,+384773RDNG#

KT2700 initialization complete.


In [ ]:
def daq_KT2700(tp, filename, save_path, column_names, instrument):
     while True:
        lib_KT2700.SingleDataAcquisition(tp, filename, save_path, column_names, instrument)
        
if __name__ == '__main__':
    # Create a time provider generator instance
    SAMPRINGRATE = 1
    tg = lib_time.time_generator(sampling_rate=SAMPRINGRATE)
    daq_KT2700(tg, KT2700["filename"], KT2700["save_path"], KT2700["save_path"],KT2700["instrument"])

KeyboardInterrupt: 